# 单表格取出工作事项

In [1]:
import xlwings as xw
import pandas as pd
import numpy as np
import difflib

In [2]:
path = r"D:\work\PMO\资源投入自动化\附件3_应用建设_数据组_双周计划_翁圳滨_3月.xlsx"
wb = xw.Book(path)
#表格总数
sht_ai =  len(wb.sheets)
#当前表格
sht_i = 2
sht = wb.sheets[sht_i]
sht_value = sht.range('A1:Y41').value

In [6]:
#----------------只取周工作表---------

t = [l for l in wb.sheets]
sht_aname = list()
for i in range(0,sht_ai):
    sht_aname.append(t[i].name)

sht_real_aname = [i for i in sht_aname if "周工作总结" in i]
# sht_real_aname = list(filter(lambda x:x.find("周工作总结") >= 0,sht_aname))
# sht_aname_index = [i for i in sht_aname if "周工作总结" in i]
sht_aname_index = list()
for i in sht_aname:
    if "周工作总结" in i: sht_aname_index.append(True)
    else:sht_aname_index.append(False)
sht_aname_index
sht = wb.sheets[sht_real_aname[0]]
#取表格使用范围
# sht_value = sht.range('A1:T30').value
sht_value = sht.range((1,1),sht.used_range.shape).value
sht_value
t = [i for i in range(0,sht_ai)]
t = np.array(t)[np.array(sht_aname_index)]
for i in t:
    print(i)


2
3
4
5
6


In [37]:
sht_aname_index

[False, False, True, True, True, True, True]

In [46]:
t = [i for i in range(0,sht_ai)]
t = np.array(t)[np.array(sht_aname_index)]
for i in t:
    print(i)


2
3
4
5
6


### 取出本周有工作事项

#### 行数 rnum

In [12]:
num_list  = [i[0] for i in sht_value]
frnum = num_list.index('任务编号') + 1
try:
    lrnum = num_list.index('小计') 
except:
    lrnum = num_list.index('小计（小时）') 
num_list_right =  list(filter(None, num_list[frnum:lrnum]))

# python list 取数 [0:1] 只取第一个数，第1位次的第二个值不取
# num_list[head:tail]
# [1.0, 2.0, 3.0, None]
rnum = len(num_list_right)
rnum

3

#### 列数 cnum

In [33]:

if sht_i == 2:
    l  = sht_value[1]
    l = ['None' if v is None else v for v in l]

    res = difflib.get_close_matches('项目用时统计', l, 1, cutoff=0.5)

    cnum = sht_value[1].index(res[0])
else:
    l  = sht_value[0]
    l = ['None' if v is None else v for v in l]

    res = difflib.get_close_matches('项目用时统计', l, 1, cutoff=0.5)
    
    cnum = sht_value[0].index(res[0])
cnum


13

#### 将本周的工作事项导出

In [6]:
worktemp = pd.DataFrame(sht_value[frnum-1:lrnum+1])
worktemp = worktemp.iloc[:,0:(cnum+1)]
rowname = worktemp.iloc[0]
worktemp = worktemp.rename(columns=rowname).iloc[1:]
worktemp.columns = [*worktemp.columns[:-1], '项目投入统计']
worktemp = worktemp.reset_index(drop=True)
worktemp

,任务编号,任务名称,负责人,协助人,交付件/工作文档,计划完成比例,实际完成比例,任务状态,星期一,星期二,星期三,星期四,星期五,项目投入统计
0,1.0,数据资产学习,翁圳滨,None,课件,1.0,1.0,完成,0.0,0.0,0.0,7.0,5.0,12.0
1,2.0,PMO,翁圳滨,None,项目管理文档,1.0,1.0,完成,2.0,7.0,6.0,0.0,2.0,17.0
2,3.0,其他任务,翁圳滨,None,资料文档,1.0,1.0,完成,5.0,0.0,1.0,0.0,0.0,6.0
3,小计（小时）,None,None,None,None,None,None,None,8.0,8.0,8.0,8.0,8.0,40.0


### 判断项目总计是否正确 _

In [160]:
# for i in range(0,rnum):
    

### 判断横纵总计是否相等

In [161]:
# dist = worktemp.iloc[-1,-1] - worktemp.iloc[:-1,-1].sum()
# dist
# if dist == 5:
    

### 最终输出工作事项

### 输出到新的excel中

In [162]:
wb0 = xw.books.open(r".\汇总.xlsx")
wb0.sheets[0].range(1,1).expand().value = worktemp.values.tolist()
wb0.save()
wb0.app.quit()

### 关闭excel

In [ ]:
wb.app.display_alerts=False
wb.app.quit()